# HPO

In [1]:
%cd ..

/Users/uribagi/Documents/GitHub/Latent-IQP


In [2]:
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
import optuna
import pennylane as qml
import jax
from jax import numpy as jnp
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from datasets.bipartites import BipartiteGraphDataset

/Users/uribagi/Documents/GitHub/Latent-IQP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nodes = 10
grid_conn = aachen_connectivity()
num_qubits = nodes * (nodes - 1) // 2
gates = efficient_connectivity_gates(grid_conn, num_qubits, 1) 
circ = iqp.IqpSimulator(num_qubits, gates, device='lightning.qubit')

train_dataset = BipartiteGraphDataset(num_samples=256, # Full samples for objective
                                      num_vertices=nodes,
                                      ratio_bipartite=1.0,
                                      edge_prob=0.2)
train_ds, _ = train_dataset.get_all_data() 
train_ds = jnp.array(train_ds)

base_key = jax.random.PRNGKey(42)


Bipartite Dataset (k=10, p=0.2, connected=True):
Target samples: 512, Generated samples: 512
Bipartite: 512, Non-Bipartite: 0
Target Ratio (Bipartite): 1.000, Actual Ratio: 1.000


In [4]:
base_sigma = median_heuristic(train_ds)
base_params_init = initialize_from_data(gates, train_ds)

In [5]:
from utils.hpo import run_hpo

In [6]:
study = run_hpo(
    iqp_circ = circ,
    base_sigma = base_sigma,
    base_params_init = base_params_init,
    train_ds = train_ds,
    n_trials = 100,
    n_iters_hpo = 250,
    n_ops = 2500,
    n_samples = 2500,
)

[I 2025-05-23 14:21:32,925] A new study created in memory with name: no-name-c35d4e5a-4e9c-40d5-8b64-037d551cb40b


Trial 0:
  Learning Rate: 0.0023061066398712316
  Sigma Multiplier: 1.9365813118345911
  Initialization Multiplier: 0.01


Training Progress: 100%|██████████| 500/500 [00:12<00:00, 40.95it/s, loss=-0.000124, elapsed time=0.02, total time=12.9]
[I 2025-05-23 14:21:45,956] Trial 0 finished with value: -0.00012370248856206212 and parameters: {'learning_rate': 0.0023061066398712316, 'sigma_multiplier': 1.9365813118345911, 'initialization_multiplier': 0.01}. Best is trial 0 with value: -0.00012370248856206212.


Training has not converged after 500 steps
Trial 0 final loss: -0.00012370
Trial 1:
  Learning Rate: 0.09476086559660529
  Sigma Multiplier: 0.10204858785512157
  Initialization Multiplier: 0.1


Training Progress: 100%|██████████| 500/500 [00:22<00:00, 22.15it/s, loss=-0.000001, elapsed time=0.04, total time=22.6]
[I 2025-05-23 14:22:08,546] Trial 1 finished with value: -1.199055560233657e-06 and parameters: {'learning_rate': 0.09476086559660529, 'sigma_multiplier': 0.10204858785512157, 'initialization_multiplier': 0.1}. Best is trial 0 with value: -0.00012370248856206212.


...

Training has not converged after 500 steps
Trial 97 final loss: -0.00029110
Trial 98:
  Learning Rate: 0.01953051677747107
  Sigma Multiplier: 1.3117244750963923
  Initialization Multiplier: 2


Training Progress: 100%|██████████| 500/500 [00:13<00:00, 36.61it/s, loss=-0.000179, elapsed time=0.03, total time=13.7]
[I 2025-05-23 14:46:11,115] Trial 98 finished with value: -0.0001794914552515593 and parameters: {'learning_rate': 0.01953051677747107, 'sigma_multiplier': 1.3117244750963923, 'initialization_multiplier': 2}. Best is trial 24 with value: -0.00033393625067007265.


Training has not converged after 500 steps
Trial 98 final loss: -0.00017949
Trial 99:
  Learning Rate: 0.010127310441447736
  Sigma Multiplier: 1.455661436653867
  Initialization Multiplier: 2


Training Progress: 100%|██████████| 500/500 [00:13<00:00, 38.10it/s, loss=-0.000307, elapsed time=0.03, total time=13.1]
[I 2025-05-23 14:46:24,258] Trial 99 finished with value: -0.00030711956069233616 and parameters: {'learning_rate': 0.010127310441447736, 'sigma_multiplier': 1.455661436653867, 'initialization_multiplier': 2}. Best is trial 24 with value: -0.00033393625067007265.


Training has not converged after 500 steps
Trial 99 final loss: -0.00030712
Best trial #24 → value: -0.000334
  params: {'learning_rate': 0.012794809953647018, 'sigma_multiplier': 0.9358811434892141, 'initialization_multiplier': 1}


In [7]:
best_hyperparams = study.best_params
best_loss_value = study.best_value

print("\nOptimization Finished!")
print(f"Best hyperparameters found: {best_hyperparams}")
print(f"Best loss value: {best_loss_value}")

NameError: name 'study' is not defined

In [ ]:
print("\nRunning full training with best hyperparameters...")

LEARNING_RATE_BEST = best_hyperparams['learning_rate']
SIGMA_MULTIPLIER_BEST = best_hyperparams['sigma_multiplier']
INITIALIZATION_MULTIPLIER_BEST = best_hyperparams['initialization_multiplier']

# Re-initialize components with best hyperparameters
sigma_val = median_heuristic(train_ds) * SIGMA_MULTIPLIER_BEST
params_init_best = initialize_from_data(gates, jnp.array(train_ds)) * INITIALIZATION_MULTIPLIER_BEST

loss_kwarg_best = {
    "params": params_init_best,
    "iqp_circuit": circ,
    "ground_truth": jnp.array(train_ds),
    "sigma": [sigma_val],
    "n_ops": 2500, 
    "n_samples": 5000, 
    "key": base_key, 
}

loss_fn_best = genq.mmd_loss_iqp
trainer_best = iqp.Trainer("Adam", loss_fn_best, stepsize=LEARNING_RATE_BEST)

print(f"Training with: LR={LEARNING_RATE_BEST:.6f}, SigmaM={SIGMA_MULTIPLIER_BEST:.4f}, InitM={INITIALIZATION_MULTIPLIER_BEST:.6f}")
history_best = trainer_best.train(n_iters=2500, loss_kwargs=loss_kwarg_best, turbo=1) # Original iterations
trained_params_best = trainer_best.final_params
final_loss_best_run = trainer_best.losses[-1]

print(f"\nFull training with best params completed.")
print(f"Final loss: {final_loss_best_run:.8f}")


Running full training with best hyperparameters...
Training with: LR=0.023320, SigmaM=1.7404, InitM=0.053727


Training Progress: 100%|██████████| 2500/2500 [00:19<00:00, 129.98it/s, loss=0.000040, elapsed time=0.01, total time=19.2] 

Training has not converged after 2500 steps

Full training with best params completed.
Final loss: 0.00003967
